In [ ]:
from posix import chdir
from google.colab import drive
drive.mount('/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment, it should be under CS254/CS254-Assignments/netid-A2
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS5990/lottery-tickets' # change this directory to yours
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
chdir(GOOGLE_DRIVE_PATH)

In [ ]:
"""
plots.ipynb

Likely move to other files after all plots confirmed to work.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 4/30/24
"""
import functools
from importlib import reload
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy.stats import norm
import tensorflow as tf

from src.harness import history
from src.metrics import experiment_aggregations as e_agg
from src.metrics import trial_aggregations as t_agg

In [ ]:
# Constants and function declarations
directory: str = 'experiments/multiprocessing_lenet/experiment_summary.pkl'
# directory: str = 'experiments/with_cached_fields/experiment_summary.pkl'

# Accuracy metrics
best_train_accuracy: callable = functools.partial(t_agg.get_best_accuracy_percent, train=True)
best_test_accuracy: callable = functools.partial(t_agg.get_best_accuracy_percent, train=False)

loss_before_training: callable = t_agg.get_loss_before_training
accuracy_before_training: callable = t_agg.get_accuracy_before_training

# Positive/negative ratio metrics
initial_weights_positive_percent_by_layer: callable = functools.partial(t_agg.get_positive_percent_of_weights_across_all_layers, use_initial=True)
initial_weights_negative_percent_by_layer: callable = functools.partial(t_agg.get_negative_percent_of_weights_across_all_layers, use_initial=True)

final_weights_positive_percent_by_layer: callable = functools.partial(t_agg.get_positive_percent_of_weights_across_all_layers, use_initial=False)
final_weights_negative_percent_by_layer: callable = functools.partial(t_agg.get_negative_percent_of_weights_across_all_layers, use_initial=False)

In [ ]:
experiment_summary = history.ExperimentSummary.load_from(directory)
experiment_summary.experiments.keys()

In [ ]:
def create_line_graph_with_confidence_intervals_over_sparsities(
    experiment_summary: history.ExperimentSummary, 
    aggregate_experiment_values: callable, 
    confidence: float = 0.95, 
    legend: str = None,
    ) -> plt.figure:
    """
    Function which creates the base line graph for an experiment summary
    and includes confidence intervals over the y-axis values.

    Args:
        experiment_summary (history.ExperimentSummary): Object with data about the experiment.
        aggregate_trial_values (callable): Function used to aggregate values for trials which will
          go on the x-axis. Often will be something like a function retrieving sparsity.
        aggregate_experiment_values (callable): Function for aggregating the values across experiments
          which will go on the y-axis. Examples would be test/train accuracy, early stopping iteration, etc.
        confidence (float, optional): Confidence level to use when plotting confidence intervals. 
          Must be between 0 and 1. Defaults to 0.95.
        legend (str, optional): Optional legend to plot the line with.

    Raises:
        ValueError: Confidence interval cannot be >= 1 or < 0.

    Returns:
        plt.figure: Matplotlib figure which additional labels can then be applied to
    """
    
    if confidence >= 1 or confidence < 0:
      raise ValueError('Confidence must be between 0 and 1')
    
    # Compute x-axis values. Will often be something like sparsity %
    sparsities: np.array = experiment_summary.aggregate_across_experiments(t_agg.get_sparsity_percentage, e_agg.mean_over_experiments)
    # Compute y-axis values aggregated over experiments along with their standard deviation
    aggregated_experiment_values: np.array = experiment_summary.aggregate_across_experiments(aggregate_experiment_values, e_agg.mean_over_experiments)
    aggregated_experiment_std: np.array = experiment_summary.aggregate_across_experiments(aggregate_experiment_values, e_agg.std_over_experiments)

    # Calculate Z-score and standard error to make confidence interval
    z_score: float = norm.ppf((1 + confidence) / 2)
    aggregated_experiment_std: np.array = np.array(aggregated_experiment_std)
    sample_count: int = len(experiment_summary.experiments)
    confidence_interval: np.array = z_score * aggregated_experiment_std  / np.sqrt(sample_count)
    
    fig = plt.figure()
    plt.plot(sparsities, aggregated_experiment_values, label=legend)
    plt.gca().invert_xaxis()
    plt.fill_between(
        sparsities, 
        aggregated_experiment_values - confidence_interval, 
        aggregated_experiment_values + confidence_interval, 
        color='gray', 
        alpha=0.3, 
        label=f'{confidence * 100:.2f}% CI'
    )
    return fig

In [ ]:
fig = create_line_graph_with_confidence_intervals_over_sparsities(experiment_summary, best_test_accuracy, legend='Test Accuracy')
# Set y-axis tick labels to be rounded to two decimal places
fig.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:0.2f}%'))
fig.gca().set_title('Best Accuracy at Early Stopping Point Over Iterative Pruning')
fig.gca().set_ylabel('Accuracy %')
fig.gca().set_xlabel('Sparsity (% Unpruned Weights)')
fig.gca().legend()
fig.gca().grid()
fig.show()

In [ ]:
fig = create_line_graph_with_confidence_intervals_over_sparsities(experiment_summary, initial_weights_positive_percent_by_layer)
fig.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:0.2f}%'))
fig.gca().set_title('Percentage of Positive Masked Initial Weights Over Iterative Pruning')
fig.gca().set_ylabel('Proportion of Positive Weights')
fig.gca().set_xlabel('Sparsity (% Unpruned Weights)')
fig.gca().legend()
fig.gca().grid()
fig.show()

In [ ]:
fig = create_line_graph_with_confidence_intervals_over_sparsities(experiment_summary, final_weights_positive_percent_by_layer)
fig.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:0.2f}%'))
fig.gca().set_title('Percentage of Positive Masked Final Weights Over Iterative Pruning')
fig.gca().set_ylabel('Percent of Positive Weights')
fig.gca().set_xlabel('Sparsity (% Unpruned Weights)')
fig.gca().legend()
fig.gca().grid()
fig.show()

In [ ]:
fig = create_line_graph_with_confidence_intervals_over_sparsities(experiment_summary, loss_before_training)
fig.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:0.2f}%'))
fig.gca().set_title('Loss from Reset/Masked Initial Weights')
fig.gca().set_ylabel('Loss')
fig.gca().set_xlabel('Sparsity (% Unpruned Weights)')
fig.gca().legend()
fig.gca().grid()
fig.show()

In [ ]:
fig = create_line_graph_with_confidence_intervals_over_sparsities(experiment_summary, accuracy_before_training)
fig.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:0.2f}%'))
fig.gca().set_title('Accuracy from Reset/Masked Initial Weights')
fig.gca().set_ylabel('Accuracy %')
fig.gca().set_xlabel('Sparsity (% Unpruned Weights)')
fig.gca().legend()
fig.gca().grid()
fig.show()